In [30]:
import ollama

ollama.pull('gemma2:27b')

response = ollama.chat(
    model='gemma2:27b',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}]
)
print(response['message']['content'])


The sky appears blue due to a phenomenon called **Rayleigh scattering**.

Here's how it works:

1. **Sunlight Enters the Atmosphere:** Sunlight is actually white, containing all colors of the rainbow. When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).
2. **Scattering Occurs:** These air molecules are much smaller than the wavelengths of visible light. As a result, they scatter the shorter wavelengths of light (blue and violet) more effectively than longer wavelengths (red, orange, yellow).
3. **Blue Light Dominates:** The scattered blue light travels in many directions, making the sky appear blue to our eyes.

**Why not Violet?**

Even though violet light is scattered even more than blue light, our eyes are less sensitive to violet wavelengths. Therefore, we perceive the sky as predominantly blue.


Let me know if you have any other questions!


In [31]:
import pandas as pd

output_dir = '../relevance'

df = pd.read_csv('../gym/data.csv')

In [1]:
# for each row of the dataframe, we will ask the model to generate a response
# We want to know if the  table is relevant to the queries we have and a score of how relevant it is to the query
# then add to each row the 10 queries as columns and the score of the model for each query


queries = {
    1: '(Graph structured) AND (visual Question Answering)',
    2: '"Autonomous Surface Vehicles" AND "Federated Learning"',
    3: 'RAG AND (Long-context LLM) OR (long context language model)',
    4: 'Applications of visual question answering systems',
    5: 'Fairness AND socials',
    6: '(Regulatory Capture AND IA OR ML) OR (Regulatory Capture OR Machine Learning)',
    7: '(Segmentation OR SAM) AND failure',
    8: '(Graph Convolutional Network) OR (GCN) AND ',
    9: '((Question Answering) AND (visual)) OR (VQA)',
    10: 'FL AND decentralised',
}

response_format = '{ "relevant": true, //boolean \n "score": 0.9, //float \n "explanation": "The quick brown fox jumps over the lazy dog" //string (only if needed and max 30 word)}'

def get_prompt(query, table):
    return (f"Given the table [{table}], how relevant is the table to the query: [{query}]?"
            f" please provide a score between 0 and 1 and true or false if the table is relevant to the query,"
            f" response must be in the JSON format: {response_format}")

def get_prompt_response(query, table):
    response = ollama.chat(
        model='gemma2',
        messages=[{'role': 'user', 'content': get_prompt(query, table)}]
    )
    return response['message']['content']


# for each row of the dataframe, we will ask the model to generate a response
# then we export to json the results 
for index, row in df.iterrows():
    for query in queries:
        try:
            response = get_prompt_response(queries[query], row)
            score, relevant = response.split('|')
            print(f"Exporting {index}_{query}.json")
            with open(f'{output_dir}/{index}_{query}.json', 'w') as f:
                f.write(response)
        except Exception as e:
            print(e)
            print(response)
            print(queries[query], row)
            break        


NameError: name 'df' is not defined